In [1]:
import pickle
import psycopg2
import math
import numpy as np
import io
import re
import sys
sys.path.append('..')
from utils.another import get_ngrams, key_dependent_dict, condition
from collections import Counter

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\persi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
conn = psycopg2.connect(host="localhost", database="postgres", user="postgres", password="igsvemina1201")
cur = conn.cursor()

In [3]:
cur.execute('SELECT count(*) from unigram_counter')
vocab_size = cur.fetchone()[0]
cur.execute('SELECT sum(unigram_count) from unigram_counter')
n_unigrams = cur.fetchone()[0]
cur.execute('SELECT sum(bigram_count) from bigram_counter')
n_bigrams = cur.fetchone()[0]
cur.execute('SELECT sum(trigram_count) from trigram_counter')
n_trigrams = cur.fetchone()[0]

print(f'vocab size - {vocab_size} \
      \nnumber of ngrams in train corpus:\nungirams - {n_unigrams}\nbigrams - {n_bigrams}\ntrigrams - {n_trigrams}')

vocab size - 2183895       
number of ngrams in train corpus:
ungirams - 829250940
bigrams - 798949912
trigrams - 768648884


In [5]:
test_file_path='../data/test_v2.txt'
test_unigram_counter = Counter()
test_bigram_counter = Counter()
test_trigram_counter = Counter()
with io.open(test_file_path, encoding='utf-8', mode='rt') as test_file:
    for line in test_file:
        comma_pos = line.find(',')
        line = line[comma_pos+1:].rstrip()
        line = line[1:-1] # skip "
        sentence = ["<s>"] + line.lower().split(' ') + ["</s>"]
        test_unigram_counter.update(sentence)
        test_bigram_counter.update(get_ngrams(sentence, 2))
        test_trigram_counter.update(get_ngrams(sentence, 3))
with open('../data/test_unigram_counter.csv', 'w', encoding='utf-8') as unigram_counter_csv, \
    open('../data/test_bigram_counter.csv', 'w', encoding='utf-8') as bigram_counter_csv, \
    open('../data/test_trigram_counter.csv', 'w', encoding='utf-8') as trigram_counter_csv:
    unigram_counter_csv.write('first_gram,unigram_count\n')
    for unigram, count in test_unigram_counter.items():
        unigram_counter_csv.write('{} {}\n'.format(unigram, count))
    bigram_counter_csv.write('first_gram,second_gram,bigram_count\n')
    for bigram, count in test_bigram_counter.items():
        bigram_counter_csv.write('{} {}\n'.format(' '.join(bigram), count))
    trigram_counter_csv.write('first_gram,second_gram,third_gram,trigram_count\n')
    for trigram, count in test_trigram_counter.items():
        trigram_counter_csv.write('{} {}\n'.format(' '.join(trigram), count))
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS test_unigram_counter')
cur.execute('DROP TABLE IF EXISTS test_bigram_counter')
cur.execute('DROP TABLE IF EXISTS test_trigram_counter')
cur.execute('''CREATE TABLE test_unigram_counter( first_gram VARCHAR, unigram_count INT)''')
cur.execute('''CREATE TABLE test_bigram_counter( first_gram VARCHAR, second_gram VARCHAR, bigram_count INT)''')
cur.execute('''CREATE TABLE test_trigram_counter( first_gram VARCHAR, second_gram VARCHAR, \
            third_gram VARCHAR, trigram_count INT)''')
cur.execute("COPY test_unigram_counter FROM 'C:/HSE/3_module/BWI_project/data/test_unigram_counter.csv' DELIMITER ' ' CSV HEADER")
cur.execute("COPY test_bigram_counter FROM 'C:/HSE/3_module/BWI_project/data/test_bigram_counter.csv' DELIMITER ' ' CSV HEADER")
cur.execute("COPY test_trigram_counter FROM 'C:/HSE/3_module/BWI_project/data/test_trigram_counter.csv' DELIMITER ' ' CSV HEADER")
conn.commit()
cur.close()

In [6]:
cur = conn.cursor()
# extract test unigram count from train table
cur.execute("SELECT * FROM unigram_counter as T WHERE T.first_gram in (SELECT first_gram FROM test_unigram_counter)")
train_unigram_counter = dict(cur.fetchall())
cur.execute("SELECT * FROM bigram_counter as T WHERE (T.first_gram,T.second_gram) in (SELECT first_gram,second_gram FROM test_bigram_counter)")
train_bigram_counter = dict(map(lambda x: ((x[0],x[1]),x[2]), cur.fetchall()))
cur.execute("SELECT * FROM trigram_counter as T WHERE (T.first_gram,T.second_gram,T.third_gram) in (SELECT first_gram,second_gram,third_gram FROM test_trigram_counter)")
train_trigram_counter = dict(map(lambda x: (x[:-1],x[-1]), cur.fetchall()))

In [24]:
def encode_line(line):
    comma_pos = line.find(',')
    sentence = line[comma_pos+1:].rstrip()
    sentence = sentence[1:-1] # skip "
    return line[:comma_pos+1], sentence

def decode_sentence(sentence):
    return '"' + sentence + '"'+'\n'

In [207]:
# new Lm
class InterpolationLanguageModel:
    def __init__(self, vocab_size, n=2, lambdas=[0.5,0.5]):
        assert len(lambdas) == n
        assert sum(lambdas) == 1.0
        assert n==2 or n==3
        self.n = n
        self.lambdas = {(i+1):l for i, l in enumerate(lambdas)}        
        self.vocab_size = vocab_size

    def fit(self, train_counters, n_ngrams):  
        ngrams_statistics = {1:{'total':n_ngrams[1], 'counter':train_counters[1]}, \
                                  2:{'total':n_ngrams[2], 'counter':train_counters[2]}, \
                                  3:{'total':n_ngrams[3], 'counter':train_counters[3]}}
        # ADD-ONE SMOOTHING
        v = self.vocab_size
        for n in range(1,4):
            if n == 1:
                # probabilities for unseen ngrams
                N = ngrams_statistics[n]['total']
                compute_p_unseen = lambda x : (ngrams_statistics[n]['counter'].get(x,0) + 1) / (N + v)
                ngrams_statistics[n]['probs'] = key_dependent_dict(compute_p_unseen)
                # probabilities for known ngrams
                for word, count in ngrams_statistics[n]['counter'].items():
                    ngrams_statistics[n]['probs'][word] = (count + 1) / (N + v)                               
            else:
                ngrams_statistics[n]['probs'] = dict()
                # probabilities for known ngrams
                for ngram, count in ngrams_statistics[n]['counter'].items():
                    context, word = condition(ngram), ngram[-1]
                    condition_count = ngrams_statistics[n-1]['counter'][context]
                    if context not in ngrams_statistics[n]['probs']:
                        ngrams_statistics[n]['probs'][context] = dict()
                    ngrams_statistics[n]['probs'][context][word] = (count + 1) / (condition_count + v)           
        self.ngrams_statistics = ngrams_statistics
    
    def prob(self, ngram):
        if isinstance(ngram,str):
            return self.ngrams_statistics[1]['probs'][ngram]
        n = len(ngram)
        context, word = condition(ngram), ngram[-1]
        # check if ngram is unseen
        if context not in self.ngrams_statistics[n]['probs']:
            self.ngrams_statistics[n]['probs'][context] = dict()
        if word not in self.ngrams_statistics[n]['probs'][context]:
            self.ngrams_statistics[n]['probs'][context][word] = (self.ngrams_statistics[n]['counter'].get(ngram,0) + 1) / \
                          (self.ngrams_statistics[n-1]['counter'].get(context,0) + self.vocab_size)
        return self.ngrams_statistics[n]['probs'][context][word]
            
    def get_prob(self, ngram):
        p = 0        
        for n in range(self.n,0,-1):            
            p += self.lambdas[n] * self.prob(ngram)
            ngram = ngram[-1] if n==2 else ngram[1:]     
        return p
            
    def test(self, ngram_counter):
        assert len(list(ngram_counter.keys())[0]) == self.n
        perplexity = 0             
        n_ngrams = sum(ngram_counter.values())
        for ngram, count in ngram_counter.items():
            perplexity += count * math.log2(self.get_prob(ngram))
        perplexity = math.pow(2, -1*(perplexity/n_ngrams))        
        return perplexity
    
    def insert_missing_word(self, sentence):
        tokenized_sentence = ["<s>"] + sentence.lower().split() + ["</s>"]
        # slice ngrams under consideration
        slicer = slice(1,-1) if self.n == 2 else slice(0,-2)
        sentence_ngrams = list(get_ngrams(tokenized_sentence, self.n))[slicer]
        spaces_inds = [m.start() for m in re.finditer(' ', sentence)]
        minP_ngram_arg = np.argmin([self.get_prob(ngram) for ngram in sentence_ngrams])
        minP_ngram = sentence_ngrams[minP_ngram_arg]
        space_ind = spaces_inds[minP_ngram_arg]
        context, word = condition(minP_ngram), minP_ngram[-1]
        candidates = self.ngrams_statistics[self.n]['probs'][context]
        candtidates_probs = {c:prob*self.get_prob((c,word)) for c, prob in candidates.items()}
        missing_word = max(candtidates_probs, key=candtidates_probs.get)
        outline = sentence[:space_ind+1] + missing_word + sentence[space_ind:]
        return outline
        
    def fill_gaps(self, test_file_path, submission_file_path='submission.txt'):
        with io.open(test_file_path, encoding='utf-8', mode='rt') as infile, \
            io.open(submission_file_path, encoding='utf-8', mode='wt') as outfile:
            outfile.write(infile.readline()+'\n')
            for i, line in enumerate(infile):
#                 if (i+1) % 1000 == 0:
                print(f'working on {i+1} file', end='\r')
                n_line, sentence = encode_line(line)
                sentence = self.insert_missing_word(sentence)
                outline = n_line + decode_sentence(sentence)
                outfile.write(outline)

In [208]:
train_counters = {1:train_unigram_counter, 2:train_bigram_counter, 3:train_trigram_counter}
n_ngrams = {1:n_unigrams, 2:n_bigrams, 3:n_trigrams}
lm = InterpolationLanguageModel(vocab_size=vocab_size)
lm.fit(train_counters, n_ngrams)

In [209]:
print(f'perplexity = {lm.test(ngram_counter=test_bigram_counter):.2f}')

perplexity = 1038.58


In [ ]:
lm.fill_gaps(test_file_path=test_file_path, submission_file_path='../data/submission.txt')

# draft

#### smoothing

In [23]:
from sklearn.linear_model import LinearRegression
uni_coc = Counter(sorted(train_unigram_counter.values()))
log_r = np.log10(list(uni_coc.keys())).reshape(-1, 1)
log_Z = np.log10(list(uni_coc.values())).reshape(-1, 1)
reg = LinearRegression().fit(log_r, log_Z)
a, b = reg.intercept_, reg.coef_[0]

In [24]:
a, b

(array([2.55806461]), array([-0.57176488]))

In [197]:
#finds the slope for the best fit line
def findBestFitSlope(x,y):
    m = (( mean(x)*mean(y) - mean(x*y) ) / 
          ( mean(x)** 2 - mean(x**2)))

    return m
      
#finds the intercept for the best fit line
def findBestFitIntercept(x,y,m):
    c = mean(y) - m*mean(x)
    return c

In [ ]:
log_r = np.log10(r).reshape(-1, 1)
log_Z = np.log10(Z).reshape(-1, 1)
reg = LinearRegression().fit(log_r, log_Z)
a, b = reg.intercept_, reg.coef_[0]
S = np.power(10, a+b*np.log(np.arange(1, np.max(r)+2)))